In [ ]:
import numpy as np
import pandas as pd
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.model_selection import RandomizedSearchCV
import spacy
import string
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

# Cargar el modelo de spaCy para características lingüísticas
nlp = spacy.load('en_core_web_sm')


In [ ]:
def word2features(sent, i):
    """
    Extrae características avanzadas para cada palabra en la secuencia
    """
    word = sent[i][0]
    
    # Análisis con spaCy
    doc = nlp(word)
    token = doc[0]
    
    # Características básicas
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word.has_hyphen': '-' in word,
        'word.has_digit': any(char.isdigit() for char in word),
        'word.has_punct': any(char in string.punctuation for char in word),
        
        # Características lingüísticas de spaCy
        'pos': token.pos_,
        'dep': token.dep_,
        'is_stop': token.is_stop,
        'is_alpha': token.is_alpha,
        'is_punct': token.is_punct,
        'like_num': token.like_num,
        'shape': token.shape_,
    }
    
    # Características de la palabra anterior
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.has_digit': any(char.isdigit() for char in word1),
            '-1:word.has_punct': any(char in string.punctuation for char in word1),
        })
    else:
        features['BOS'] = True
    
    # Características de la palabra siguiente
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:word.has_digit': any(char.isdigit() for char in word1),
            '+1:word.has_punct': any(char in string.punctuation for char in word1),
        })
    else:
        features['EOS'] = True
        
    # Características de ventana
    if i > 1:
        word2 = sent[i-2][0]
        features['-2:word.lower()'] = word2.lower()
    if i < len(sent)-2:
        word2 = sent[i+2][0]
        features['+2:word.lower()'] = word2.lower()
    
    return features


In [ ]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

# Convertir los datos al formato para CRF
X_train = [sent2features(s) for s in train_data]
y_train = [sent2labels(s) for s in train_data]

X_test = [sent2features(s) for s in test_data]
y_test = [sent2labels(s) for s in test_data]


In [ ]:
# Definir el espacio de hiperparámetros para la búsqueda
params_space = {
    'c1': np.logspace(-3, 3, 7),  # Coeficiente de regularización L1
    'c2': np.logspace(-3, 3, 7),  # Coeficiente de regularización L2
    'max_iterations': [50, 100, 150],
    'feature.possible_transitions': [True, False],
    'feature.possible_states': [True, False]
}

# Crear el modelo CRF base
crf = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)

# Realizar búsqueda aleatoria de hiperparámetros con validación cruzada
rs = RandomizedSearchCV(
    estimator=crf,
    param_distributions=params_space,
    n_iter=20,  # Número de combinaciones a probar
    cv=3,       # Número de folds para validación cruzada
    verbose=1,
    n_jobs=-1   # Usar todos los núcleos disponibles
)

# Entrenar el modelo con búsqueda de hiperparámetros
print("Iniciando búsqueda de hiperparámetros...")
rs.fit(X_train, y_train)

# Obtener y mostrar los mejores hiperparámetros
print("\nMejores hiperparámetros encontrados:")
for param, value in rs.best_params_.items():
    print(f"{param}: {value}")

# Entrenar el modelo final con los mejores hiperparámetros
best_crf = CRF(
    algorithm='lbfgs',
    **rs.best_params_
)

print("\nEntrenando modelo final con los mejores hiperparámetros...")
best_crf.fit(X_train, y_train)


In [ ]:
# Evaluar el modelo en el conjunto de prueba
y_pred = best_crf.predict(X_test)

# Generar reporte de clasificación detallado
print("Reporte de clasificación detallado:")
print(flat_classification_report(y_test, y_pred))

# Análisis de errores
def analyze_errors(y_true, y_pred, X_test, test_data):
    errors = []
    for i, (true_seq, pred_seq, features_seq, original_sent) in enumerate(zip(y_true, y_pred, X_test, test_data)):
        for j, (true_label, pred_label) in enumerate(zip(true_seq, pred_seq)):
            if true_label != pred_label:
                word = original_sent[j][0]
                context = ' '.join([token[0] for token in original_sent[max(0,j-2):min(len(original_sent),j+3)]])
                errors.append({
                    'word': word,
                    'true_label': true_label,
                    'pred_label': pred_label,
                    'context': context,
                    'features': features_seq[j]
                })
    return errors

print("\nAnálisis de errores:")
errors = analyze_errors(y_test, y_pred, X_test, test_data)
error_df = pd.DataFrame(errors)

print("\nDistribución de errores por tipo de entidad:")
error_counts = error_df.groupby(['true_label', 'pred_label']).size().unstack(fill_value=0)
print(error_counts)

# Guardar los errores en un archivo CSV para análisis posterior
error_df.to_csv('error_analysis.csv', index=False)
print("\nAnálisis de errores guardado en 'error_analysis.csv'")


In [ ]:
# Función para predecir entidades en texto nuevo
def predict_entities(text, crf_model):
    # Tokenizar el texto
    tokens = nltk.word_tokenize(text)
    # Crear una secuencia de características
    sent = [(token, 'O') for token in tokens]  # 'O' es un placeholder
    X = sent2features(sent)
    
    # Predecir las etiquetas
    y_pred = crf_model.predict([X])[0]
    
    # Extraer las entidades
    entities = []
    current_entity = None
    
    for i, (token, label) in enumerate(zip(tokens, y_pred)):
        if label.startswith('B-'):
            if current_entity:
                entities.append(current_entity)
            current_entity = {'text': token, 'type': label[2:], 'start': i}
        elif label.startswith('I-') and current_entity and label[2:] == current_entity['type']:
            current_entity['text'] += ' ' + token
        else:
            if current_entity:
                entities.append(current_entity)
            current_entity = None
    
    if current_entity:
        entities.append(current_entity)
    
    return entities

# Probar el modelo con algunas oraciones de ejemplo
test_sentences = [
    "The Thai restaurant on Main Street has great pad thai for $12.99",
    "I had dinner at Le Bernardin last night and the service was excellent",
    "The prices at Sushi Express are very reasonable, especially their lunch special"
]

print("Ejemplos de predicciones del modelo CRF:")
for sentence in test_sentences:
    print(f"\nTexto: {sentence}")
    entities = predict_entities(sentence, best_crf)
    print("Entidades encontradas:")
    for entity in entities:
        print(f"- {entity['text']}: {entity['type']}")
